In [36]:
#all imports
import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
import glob
import os
try:
    sys.path.append(glob.glob('./carla/dist/carla-0.9.14-py3.7-win-amd64.egg')[0])
except IndexError:
    print('Couldn\'t import Carla egg properly')
import carla
try:
    sys.path.insert(0,r'C:\Users\ashokkumar\source\repos\AD\carla')
except IndexError:
    pass

from carla import ColorConverter as cc

from scipy.spatial import KDTree

SECONDS_PER_EPISODE = 25

SECONDS_PER_EPISODE = 10  # seconds per episode maybe needs to be removed later
FIXED_DELTA_SECONDS = 0.02 # seconds per frame
SHOW_PREVIEW = True # Show preview for debugging
NO_RENDERING = False # No rendering for training
SYNCRONOUST_MODE = False # Synchronous mode for training
SHOW_CAM = SHOW_PREVIEW     # render camera



In [37]:
## INITIALIZE THE SIMULATOR

# connect to the sim 
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
world = client.load_world('Town02')

settings = world.get_settings()
settings.no_rendering_mode = NO_RENDERING
settings.synchronous_mode = SYNCRONOUST_MODE
settings.fixed_delta_seconds = FIXED_DELTA_SECONDS
world.apply_settings(settings)
blueprint_library = world.get_blueprint_library()
actor_list = []
vehicle = None
collision_hist = []

# get map look at the map
if SHOW_CAM:
    spectator = world.get_spectator()

town_map = world.get_map()
roads = town_map.get_topology()


In [12]:

blueprint_library = world.get_blueprint_library()
model_3 = blueprint_library.find('vehicle.tesla.model3')

In [5]:
spectator.set_transform(carla.Transform(carla.Location(x=193.779999, y=171.289993, z=0.500000), carla.Rotation(pitch=-90)))


In [38]:
spawn_points = world.get_map().get_spawn_points()
for i, spawn_point in enumerate(spawn_points):
    # Draw in the spectator window the spawn point index
    world.debug.draw_string(spawn_point.location, str(i), life_time=100)
    world.debug.draw_arrow(spawn_point.location, spawn_point.location + spawn_point.get_forward_vector(), life_time=100)


In [26]:
vehicle.destroy()
vehicle = world.spawn_actor(model_3, spawn_points[12])

Current waypoint: Location(x=193.682220, y=172.089539, z=0.000000)
Target waypoint: Location(x=193.681534, y=169.089539, z=0.000000)

In [27]:
world.debug.draw_string(carla.Location(x=193.682220, y=172.089539, z=0.000000), 'O', draw_shadow=False,
        color=carla.Color(255, 0, 0), life_time=50, persistent_lines=True)

world.debug.draw_string(carla.Location(x=193.681534, y=169.089539, z=0.000000), 'O', draw_shadow=False,
        color=carla.Color(255, 0, 0), life_time=50, persistent_lines=True)


In [31]:
spectator.set_transform(carla.Transform(carla.Location(x=25.530020, y=105.549988, z=0.500000), carla.Rotation(pitch=-90)))


In [33]:
vehicle_1 = world.spawn_actor(model_3, spawn_points[28])


world.debug.draw_string(carla.Location(x=193.682220, y=172.089539, z=0.000000), 'O', draw_shadow=False,
        color=carla.Color(255, 0, 0), life_time=50, persistent_lines=True)

world.debug.draw_string(carla.Location(x=193.681534, y=169.089539, z=0.000000), 'O', draw_shadow=False,
        color=carla.Color(255, 0, 0), life_time=50, persistent_lines=True)

In [35]:
spawn_points = world.get_map().get_spawn_points()

for i, spawn_point in enumerate(spawn_points):
    # Draw in the spectator window the spawn point index
    world.debug.draw_string(spawn_point.location, str(i), life_time=100)
    # We can also draw an arrow to see the orientation of the spawn point
    # (i.e. which way the vehicle will be facing when spawned)
    world.debug.draw_arrow(spawn_point.location, spawn_point.location + spawn_point.get_forward_vector(), life_time=100)



In [ ]:
def calculate_relative_yaw(vehicle_transform, waypoint):
        # Extract yaw angles from vehicle and waypoint
        vehicle_yaw = vehicle_transform.rotation.yaw % 360
        waypoint_yaw = waypoint.transform.rotation.yaw % 360

        # Calculate the yaw difference, ensuring it's within [-180, 180] range
        yaw_diff = vehicle_yaw - waypoint_yaw

        if yaw_diff > 180:
            yaw_diff -= 360
        elif yaw_diff < -180:
            yaw_diff += 360

        return yaw_diff

### Vehicle (x=196.060120, y=243.630508, z=0.271711), Yaw:12.253863334655762, WP_Yaw:56.278587341308594 relative_yaw: -44.02472400665283

In [5]:
vehicle_yaw =  68 % 360
waypoint_yaw = -90 % 360

# Calculate the yaw difference, ensuring it's within [-180, 180] range
yaw_diff = vehicle_yaw - waypoint_yaw

if yaw_diff > 180:
    yaw_diff -= 360
elif yaw_diff < -180:
    yaw_diff += 360

print(yaw_diff)

158


: 

In [5]:
vehicle_yaw = 149.94712829589844
waypoint_yaw = -90.00496673583984 
num_lookahead = 5  # Number of waypoints to look ahead
distance_lookahead = 3.0
yaw_differences_sum = 0  # Accumulator for the sum of yaw differences
turn_direction = 1
for i in range(1, num_lookahead + 1):
    # Fetch the next waypoint 'i' steps ahead
    

    yaw_diff = 60 % 360
    yaw_diff = yaw_diff if yaw_diff <= 180 else 360 - yaw_diff
    
    # Accumulate the yaw difference
    yaw_differences_sum += yaw_diff

vehicle_yaw = vehicle_yaw % 360     
relative_yaw = (vehicle_yaw - waypoint_yaw) % 360  
print(relative_yaw)          
# Calculate relative heading as the difference between the yaw sum and the vehicle's yaw
relative_heading = (yaw_differences_sum - turn_direction*relative_yaw) % 360
print(relative_heading)
# Normalize to [-180, 180] range
if relative_heading > 180:
    relative_heading -= 360

# Adjust the heading based on the turn direction
normalized_heading = turn_direction * (relative_heading / 180.0)
print(normalized_heading)
print(np.clip(normalized_heading, -1, 1))


354.89363861083984
305.10636138916016
-0.30496465894911023
-0.30496465894911023


## Train